In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import os
import pandas as pd
import numpy as np
import sys
from tqdm import tqdm
#sys.path.append('/home/jma819/post_cmfe_analysis')
sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import dlc_utils
from matplotlib import pyplot as plt

In [9]:
from importlib import reload
reload(dlc_utils)

<module 'dlc_utils' from '/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/dlc_utils.py'>

In [4]:
dlc_directory = '/volumes/My_Passport/dlc_analysis/behavcamvideos/rotarod/3_11_2020/'
#dlc_directory = '/projects/p30771/dlc_analysis/behavcamvideos/rotarod/3_17_2020/'

In [13]:
experiment_folders = [folder for folder in os.listdir(dlc_directory) if '_M' in folder]

In [14]:
experiment_folders[1:]

['H19_M51_S4',
 'H19_M55_S57',
 'H20_M0_S57',
 'H20_M11_S42',
 'H20_M17_S28',
 'H20_M22_S56',
 'H20_M28_S4',
 'H20_M33_S18',
 'H20_M6_S41']

In [7]:
# GRIN018_H17_M41_S43
#experiment_folders = ['H19_M46_S0']

In [15]:
for folder in tqdm(experiment_folders[1:]):
    output_directory = dlc_directory+folder+'/'
    session_name = folder
    print(folder)
    dlc_output_files = os.listdir(output_directory)
    dlc_output_files.sort()
    
    names = []
    dlc_dfs = []
    coordinates_dfs = []
    for output_file in dlc_output_files:
        if output_file.endswith('000.h5'):
            print(output_file)
            dlc_output = pd.read_hdf(output_directory + output_file)
            dlc_output = dlc_utils.calculate_centroid(dlc_output)
            dlc_output.head()
            coordinates_delta_df = dlc_utils.difference_df(dlc_output)
            df_columns = list(dlc_output.columns)
            print('calculating velocity')
            velocity_df = pd.DataFrame(np.transpose(np.array([np.array([dlc_utils.velocity(coordinates_delta_df[body_part]['x'].values[frame],coordinates_delta_df[body_part]['y'].values[frame]) for frame in range(len(coordinates_delta_df))]) for body_part in list(set([df_columns[item][1] for item in range(len(df_columns))]))])), 
                                       columns=list(set([df_columns[item][1] for item in range(len(df_columns))]))) 
            for body_part in list(set([df_columns[item][1] for item in range(len(df_columns))])):
                velocity_df[body_part+'_likelihood']=dlc_output[df_columns[0][0]][body_part]['likelihood'].values
            
            names.append(output_file)
            dlc_dfs.append(velocity_df)
            coordinates_dfs.append(dlc_output)
            
    session_compiled = pd.concat(dlc_dfs, keys=names)
    coordinates_dfs_compiled = pd.concat(coordinates_dfs, keys=names)
   

 
    #save compiled session 
    session_compiled.to_hdf(output_directory+session_name+'_compiled_session_05122020.h5', key='df')
    coordinates_dfs_compiled.to_hdf(output_directory+session_name+'_compiled_coordinates_session_05122020.h5', key='df')
    
    #create behavior data aligned to the msCam frames
    aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+session_name+'_compiled_session_05122020.h5', output_directory+'timestamp.dat', 1, 0)
    aligned.reset_index(inplace=True)
    aligned['msCamFrame'] = aligned.index.values
    aligned = aligned.set_index(pd.to_timedelta(np.linspace(0, len(aligned)*(1/20), len(aligned)), unit='s'), drop=False)

    #do same for coordinates df 
    coordinates_aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+session_name+'_compiled_coordinates_session_05122020.h5', output_directory+'timestamp.dat', 1, 0)
    coordinates_aligned.reset_index(inplace=True)
    coordinates_aligned['msCamFrame'] = coordinates_aligned.index.values
    coordinates_aligned = coordinates_aligned.set_index(pd.to_timedelta(np.linspace(0, len(coordinates_aligned)*(1/20), len(coordinates_aligned)), unit='s'), drop=False)
    
    #smooth data by downsampling and interpolating 
    interpolated = dlc_utils.downsample_and_interpolate(aligned, '.05S', '1S', 'linear')
    interpolated.to_csv(output_directory+session_name+'_dlc_tracking_foranalysis_05122020.csv')

    coordinates_aligned_interpolated = dlc_utils.downsample_and_interpolate(coordinates_aligned, '.05S', '1S', 'linear')
    coordinates_aligned_interpolated.to_csv(output_directory+session_name+'_dlc_coordinates_smoothed_tracking_foranalysis_05122020.csv')
    
    coordinates_aligned.to_csv(output_directory+session_name+'_dlc_coordinates_tracking_foranalysis_05122020.csv')

  0%|          | 0/9 [00:00<?, ?it/s]

H19_M51_S4
behavCam1DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam2DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam3DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam4DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity



100%|██████████| 2561/2561 [00:02<00:00, 976.34it/s]

 11%|█         | 1/9 [01:56<15:30, 116.27s/it]

H19_M55_S57
behavCam1DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam2DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam3DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam4DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity



100%|██████████| 3388/3388 [00:03<00:00, 951.14it/s]

 22%|██▏       | 2/9 [03:44<13:17, 113.94s/it]

...


100%|██████████| 4370/4370 [00:04<00:00, 1005.97it/s]

 78%|███████▊  | 7/9 [16:21<04:41, 140.75s/it]

H20_M33_S18
behavCam1DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam2DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam3DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam4DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam5DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam6DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity



100%|██████████| 5993/5993 [00:06<00:00, 966.61it/s] 

 89%|████████▉ | 8/9 [19:14<02:30, 150.48s/it]

H20_M6_S41
behavCam1DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam2DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam3DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam4DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam5DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity



100%|██████████| 4465/4465 [00:04<00:00, 899.19it/s] 

100%|██████████| 9/9 [21:36<00:00, 144.08s/it]


In [11]:
aligned

,level_0,index,rotarod_base_l,left_r_paw,miniscope_light,rotarod_base_r,right_ear,centroid,right_f_paw,right_r_paw,...,miniscope_light_likelihood,rotarod_base_r_likelihood,right_ear_likelihood,centroid_likelihood,right_f_paw_likelihood,right_r_paw_likelihood,left_f_paw_likelihood,left_ear_likelihood,tail_base_likelihood,msCamFrame
00:00:00,1,1,0.535089,0.591663,0.459772,0.166536,0.350769,0.118243,0.082549,0.622474,...,0.002331,0.320716,0.001910,0.333397,0.002050,1.000000,0.002076,0.002061,0.999997,0
00:00:00.050025,1,1,0.535089,0.591663,0.459772,0.166536,0.350769,0.118243,0.082549,0.622474,...,0.002331,0.320716,0.001910,0.333397,0.002050,1.000000,0.002076,0.002061,0.999997,1
00:00:00.100050,3,3,0.250131,0.657130,0.343791,0.431315,0.115691,54.668786,273.099537,0.078145,...,0.001826,0.336440,0.001541,0.334698,0.001651,1.000000,0.001702,0.001688,0.999996,2
00:00:00.150075,4,4,0.364834,0.467867,0.254959,0.220168,270.869048,27.007612,0.216589,2.156649,...,0.002109,0.214833,0.001794,0.322679,0.001924,1.000000,0.001836,0.001845,0.999996,3
00:00:00.200101,6,6,1.225403,0.120657,0.147406,0.193514,0.149331,0.282554,0.070699,2.601249,...,0.001751,0.390554,0.001480,0.340071,0.001600,1.000000,0.001605,0.001572,0.999996,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00:01:38.649898,2959,959,0.412471,0.323427,0.182455,0.247635,0.102865,27.481064,0.229817,275.065200,...,0.002203,0.995264,0.002115,0.201806,0.002106,0.009314,0.001678,0.002015,0.001500,1972
00:01:38.699924,2961,961,0.304446,0.276685,0.133718,0.455044,0.080142,0.752337,0.208753,7.535320,...,0.002285,0.997835,0.002191,0.201795,0.002170,0.005944,0.001739,0.002108,0.001830,1973
00:01:38.749949,2963,963,0.774279,0.425856,0.212908,0.431984,0.285994,26.425072,0.220615,263.078910,...,0.001807,0.993912,0.001692,0.201239,0.001714,0.007593,0.001381,0.001640,0.001123,1974
00:01:38.799974,2964,964,0.379232,0.102787,0.222350,0.233284,0.261461,26.551374,0.204893,251.787437,...,0.001928,0.997678,0.001792,0.201333,0.001847,0.003843,0.001485,0.001785,0.001375,1975


In [ ]:
coordinates_dfs_compiled = pd.concat(coordinates_dfs, keys=names)

In [ ]:
coordinates_dfs_compiled

In [ ]:
coordinates_aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+session_name+'_compiled_coordinates_session_04142020.h5', output_directory+'timestamp.dat')

In [ ]:
coordinates_aligned

In [ ]:
coordinates_aligned['msCamFrame'] = coordinates_aligned.index.values

In [ ]:
interpolated = dlc_utils.downsample_and_interpolate(aligned, '.05S', '1S', 'linear')

In [ ]:
coordinates_aligned = coordinates_aligned.set_index(pd.to_timedelta(np.linspace(0, len(coordinates_aligned)*(1/20), len(coordinates_aligned)), unit='s'), drop=False)

In [ ]:
coordinates_aligned_interpolated = dlc_utils.downsample_and_interpolate(coordinates_aligned, '.05S', '1S', 'linear')

In [ ]:
coordinates_aligned_interpolated

In [ ]:
plt.plot(coordinates_aligned_interpolated['DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000']['right_r_paw']['x'].values)

In [ ]:
plt.plot(coordinates_aligned['DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000']['right_r_paw']['y'].values)

In [ ]:
plt.plot(coordinates_aligned['DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000']['right_r_paw']['x'].values)

In [ ]:
interpolated = dlc_utils.downsample_and_interpolate(aligned, '.05S', '1S', 'linear')

In [ ]:
interpolated

In [ ]:
plt.plot(interpolated['tail_base_likelihood'].values)

In [ ]:
plt.plot(interpolated['tail_base'].values)

In [ ]:
plt.plot(interpolated['right_r_paw_likelihood'].values)

In [ ]:
plt.plot(interpolated['right_r_paw'].values)

In [ ]:
#aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+'GRIN012_H17_M46_S35_compiled_session_04142020.h5', output_directory+'timestamp.dat')
dlc_analysis = pd.read_hdf(output_directory+'GRIN012_H17_M46_S35_compiled_session_04142020.h5')

In [ ]:
frame_clock_df = pd.read_table(output_directory+'timestamp.dat')

In [ ]:
dlc_full = dlc_full.reset_index()

In [ ]:
msCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 0].set_index('frameNum')
behavCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 1].set_index('frameNum')
msCam_timestamps['sysClock'][1] = 0
behavCam_timestamps['sysClock'][1] = 0

In [ ]:
msCam_timestamps

In [ ]:
behavCam_timestamps

In [ ]:
msCam_timestamps = dlc_utils.align_behavior_data(msCam_timestamps, behavCam_timestamps)
msCam_timestamps.reset_index(inplace=True)